1. Connect Jupyter Notebook to database file

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

2. Obtain information on database

In [5]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


3. Return the first 5 rows of the `facts` table in database

In [6]:
%%sql
SELECT *
FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


4. Summary statistics for: min & max population, min & max population growth

In [7]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


5. Determine which country has min & max population

In [8]:
%%sql
SELECT name, population
FROM facts
WHERE population = (SELECT MIN(population)
                   FROM facts);

Done.


name,population
Antarctica,0


In [9]:
%%sql
SELECT name, population
FROM facts
WHERE population = (SELECT MAX(population)
                   FROM facts);

Done.


name,population
World,7256490011


6. Recalculate summary statistics for min/max population & population growth excluding World population

In [10]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE population < (SELECT MAX(population)
                   FROM facts)

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


7. Calculate average value for `population` and `area`

In [13]:
%%sql
SELECT AVG(population) 'avg_population', AVG(area) 'avg_area'
FROM facts

Done.


avg_population,avg_area
62094928.32231405,555093.546184739


8. Return list of countries that have above average `population` and below average `area`

In [14]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts)
AND area < (SELECT AVG(area)
           FROM facts);

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


9. Which country has the most people?

In [17]:
%%sql
SELECT name, population
FROM facts
WHERE population < (SELECT MAX(population)
                   FROM facts)
ORDER BY population DESC
LIMIT 1

Done.


name,population
China,1367485388


10. Which country has the highest growth rate? 

In [18]:
%%sql
SELECT name, population_growth
FROM facts
ORDER BY population_growth DESC
LIMIT 1

Done.


name,population_growth
South Sudan,4.02


11. Which countries have the highest ratio of water to land? (Top 10)

In [30]:
%%sql
SELECT name, ROUND(CAST(area_water AS float)/CAST(area_land AS float), 4) 'water_land_ratio'
FROM facts
ORDER BY water_land_ratio DESC
LIMIT 10

Done.


name,water_land_ratio
British Indian Ocean Territory,905.6667
Virgin Islands,4.5202
Puerto Rico,0.5548
"Bahamas, The",0.3866
Guinea-Bissau,0.2847
Malawi,0.2594
Netherlands,0.2257
Uganda,0.2229
Eritrea,0.1644
Liberia,0.1562


12. Which countries have more water than land?

In [32]:
%%sql
SELECT name, area_water, area_land
FROM facts
WHERE area_water > area_land
ORDER BY area_water
LIMIT 10

Done.


name,area_water,area_land
Virgin Islands,1564,346
British Indian Ocean Territory,54340,60


13. Which countries will add the most people to their population next year? (Top 10)

In [35]:
%%sql
SELECT name, (birth_rate*population)/1000 'births_number'
FROM facts
WHERE population < (SELECT MAX(population)
                   FROM facts)
ORDER BY births_number DESC
LIMIT 10

Done.


name,births_number
India,24470648.6672
China,17079892.496120002
Nigeria,6833995.78784
European Union,5242284.339
Pakistan,4495358.425259999
Indonesia,4280214.22928
United States,4013897.11136
Ethiopia,3707091.07413
Bangladesh,3571766.7293000002
Brazil,2953596.88152


14. Which countries havea  higher death rate than birth rate?

In [43]:
%%sql
SELECT name, death_rate, birth_rate, ROUND(death_rate-birth_rate, 2) 'death_birth_dif'
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_birth_dif DESC

Done.


name,death_rate,birth_rate,death_birth_dif
Bulgaria,14.44,8.92,5.52
Serbia,13.66,9.08,4.58
Latvia,14.31,10.0,4.31
Lithuania,14.27,10.1,4.17
Ukraine,14.46,10.72,3.74
Hungary,12.73,9.16,3.57
Germany,11.42,8.47,2.95
Slovenia,11.37,8.42,2.95
Romania,11.9,9.14,2.76
Croatia,12.18,9.45,2.73


15. What countries have the highest population/area ratio?

In [44]:
%%sql
SELECT name, population/area 'pop_area_ratio'
FROM facts
ORDER BY pop_area_ratio DESC
LIMIT 10

Done.


name,pop_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299
